In [1]:
import os 
import pandas as pd
import numpy as np
import regex
import json

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
from sklearn.metrics import classification_report

In [3]:
langs = ["bi","cbk-zam","jam", "pih", "tpi"]

# Evaluation

In [4]:
results_dir = "../ZS_BERT/output/01"

In [6]:
def get_report(lang):
    with open(f"{results_dir}/{lang}.json") as f:
        fr = json.load(f)
    print(lang, len(fr))
    preds =[]
    golds = []
    for item in fr:
        preds.append(item["edgeSet"]["property"])
        golds.append(item["edgeSet"]["triple"][-1])
        
    results = classification_report(golds, preds, output_dict=True)
    with open(f"../data/relation_extraction/results_report/01/{lang}.json", "w") as f:
        json.dump(results, f)

In [7]:
for lang in langs:
    get_report(lang)

bi 97
cbk-zam 97
jam 97
pih 97
tpi 97


/Users/yiyichen/anaconda3/envs/re/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yiyichen/anaconda3/envs/re/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yiyichen/anaconda3/envs/re/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [157]:
for lang in langs:
    get_report(lang)

bi 99
cbk-zam 98
jam 97
pih 100
tpi 109


/Users/yiyichen/anaconda3/envs/re/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yiyichen/anaconda3/envs/re/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yiyichen/anaconda3/envs/re/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [141]:
preds =[]
golds = []

for item in bi:
    preds.append(item["edgeSet"]["property"])
    golds.append(item["edgeSet"]["triple"][-1])

In [146]:
classification_report(golds, preds, output_dict=True)

{'P106': {'precision': 0.9230769230769231,
  'recall': 0.8,
  'f1-score': 0.8571428571428571,
  'support': 30},
 'P131': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11},
 'P1376': {'precision': 0.4,
  'recall': 0.125,
  'f1-score': 0.19047619047619047,
  'support': 16},
 'P17': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10},
 'P30': {'precision': 0.9333333333333333,
  'recall': 0.7368421052631579,
  'f1-score': 0.8235294117647058,
  'support': 19},
 'P39': {'precision': 0.47368421052631576,
  'recall': 0.9,
  'f1-score': 0.6206896551724138,
  'support': 10},
 'P495': {'precision': 0.08333333333333333,
  'recall': 0.3333333333333333,
  'f1-score': 0.13333333333333333,
  'support': 3},
 'accuracy': 0.5050505050505051,
 'macro avg': {'precision': 0.4019182571814151,
  'recall': 0.4135964912280702,
  'f1-score': 0.3750244925556429,
  'support': 99},
 'weighted avg': {'precision': 0.5738634659687292,
  'recall': 0.5050505050505051,
  'f1-score': 0.515311

# post processing relation extraction dataset 

In [54]:
folder_dir = "../data/relation_extraction/verified"
output_dir = "../data/relation_extraction/post-processed"

In [68]:
import spacy
nlp= spacy.load("xx_sent_ud_sm")

In [131]:
for lang in langs:
    file = f"{output_dir}/{lang}.csv"
    df = pd.read_csv(file)
    properties = list(set(df["Property"].tolist()))
    with open(f"../data/relation_extraction/properties/{lang}.json", "w") as f:
        json.dump(properties, f)

In [61]:
def get_entities(text):
    pattern = r"\[\[(.*?)\]\]"
    p = regex.compile(pattern)
    ents = []
    for m in p.finditer(text):
        ent = m.group()
        ents.append(ent.replace("[[", "").replace("]]", ""))
    return tuple(ents)

In [125]:
def process_one_sentence(text):
    print("*"*40)
    ents = get_entities(text)
    ent1, ent2 = ents
    print(ents)
    doc = nlp(text.replace("[[","").replace("]]",""))
    left=[]
    right=[]
    tokens = []
    for idx, token in enumerate(doc):
        print(idx, token.text)
        tokens.append(token.text)
        for ent in nlp(ent1):
            if token.text==ent.text:
                if len(left)>0:
                    if idx ==left[-1]+1:
                        left.append(idx)
                else:
                    left.append(idx)
        for ent in nlp(ent2):
            if len(left)>0:
                if idx > left[-1]:
                    if token.text==ent.text:
                        right.append(idx)
    print(left, right)
    assert left[-1] < right[0]
    return ent1, ent2, left, right, tokens
        

In [132]:
def save2json(df, lang):
    df["ent1"], df["ent2"], df["left"], df["right"], df["tokens"] = zip(*df["Sentence"].apply(process_one_sentence))
    
    ls = []
    for left, right, tokens, e1, e2, property in zip(df["left"], df["right"], df["tokens"], df["e1"], df["e2"], df["Property"]):
        d = {"tokens":tokens, 
             "edgeSet":[{"left":left, "right":right }], 
            "triple":(e1,e2,property)}
        ls.append(d)
    with open(f"{output_dir}/{lang}.json", "w") as f:
        json.dump(ls, f)

In [134]:
for lang in ["pih", "tpi"]:
    file = f"{output_dir}/{lang}.csv"
    df = pd.read_csv(file)
    save2json(df, lang)
    del df

****************************************
('William Bradley Brad Pitt', 'akter')
0 William
1 Bradley
2 Brad
3 Pitt
4 (
5 Disemba
6 18
7 ,
8 1963
9 )
10 es
11 wun
12 Merikan
13 akter
14 an
15 film
16 producer
17 .
[0, 1, 2, 3] [13]
****************************************
('Taylor Lautner', 'akter')
0 Taylor
1 Lautner
2 (
3 barn
4 Febyuweri
5 11
6 ,
7 1992
8 )
9 es
10 wun
11 Merikan
12 akter
13 .
[0, 1] [12]
****************************************
('Alexandre Borges Corrêa', 'akter')
0 Alexandre
1 Borges
2 Corrêa
3 (
4 Febyuweri
5 23
6 ,
7 1966
8 )
9 es
10 wun
11 Breselyan
12 akter
13 .
[0, 1, 2] [12]
****************************************
('Andrew Garfield', 'akter')
0 Andrew
1 Garfield
2 (
3 Orgas
4 20
5 ,
6 1983
7 )
8 es
9 wun
10 Merikan
11 akter
12 .
[0, 1] [11]
****************************************
('Lília Cabral Bertolli Figueiredo', 'aktress')
0 Lília
1 Cabral
2 Bertolli
3 Figueiredo
4 (
5 barn
6 Juulai
7 13
8 ,
9 1957
10 )
11 esa
12 Breselyan
13 aktress
14 .
[0, 1, 2, 3] [1

In [62]:
text = "[[Akaki Minashvili]] es un georgiano [[politico]]"

In [89]:
df = pd.read_csv(f"{output_dir}/bi.csv")

In [87]:
process_one_sentence(text)

0 Akaki
Akaki
Minashvili
1 Minashvili
Akaki
Minashvili
2 es
Akaki
Minashvili
3 un
Akaki
Minashvili
4 georgiano
Akaki
Minashvili
5 politico
Akaki
Minashvili


([0, 1], [5])

In [58]:
def get_one_df(file):
    df = pd.read_csv(file)
    
    df1 = df[(df["Modified"].str.contains("Y")) | (df["TripleInWD"]==False)]
    df2 = df[(df["TripleInWD"]==True)]
    df_ = pd.concat([df1, df2]).drop_duplicates()
    print(len(df), len(df_))
    print(df[df["Modified"].isna()][df["TripleInWD"]==False])
    return df_[["Property", "e1", "e2", "Sentence"]].sort_values("Property")

In [59]:
for lang in langs:
    print(lang)
    filepath = f"{folder_dir}/{lang}.csv"
    df_ = get_one_df(filepath)
    
    outputfile = f"{output_dir}/{lang}.csv"
    df_.to_csv(outputfile, index=False)

bi
99 99
Empty DataFrame
Columns: [Modified, Property, e1, e2, Sentence, Properties, TripleInWD]
Index: []
cbk-zam
98 98
Empty DataFrame
Columns: [Modified, Property, e1, e2, Sentence, Properties, TripleInWD]
Index: []
jam
103 97
   Modified Property      e1     e2   
43      NaN    P2936   Q9255   Q813  \
47      NaN      P17   Q1867  Q9192   
49      NaN      P17    Q270   Q809   
51      NaN      P17     Q90   Q150   
54      NaN      P17    Q649  Q7737   
57      NaN      P17  Q33077   Q970   

                                             Sentence        Properties   
43  [[Kiergi]] a di ofishal languij a [[Kiergistan]].  ['P17', 'P2341']  \
47  Di kiapital a [[Taipei]], ah di ofishal langui...         ['P2936']   
49  Ikiapital a [[Warsaa]], ah di ofishal languij ...           ['P37']   
51  Ikiapital a [[Paris]], ah di ofishal languij a...  ['P37', 'P2936']   
54  Di kiapital a [[Masko]], ah di ofishal languij...  ['P37', 'P2936']   
57  [[Shikomor]], di ofishal languij ina [[Kom

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_30279/307442396.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df["Modified"].isna()][df["TripleInWD"]==False])
/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_30279/307442396.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df["Modified"].isna()][df["TripleInWD"]==False])
/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_30279/307442396.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df["Modified"].isna()][df["TripleInWD"]==False])
/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_30279/307442396.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df["Modified"].isna()][df["TripleInWD"]==False])
/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_30279/307442396.py:8: UserWarning: Boolean Series key will be reindexed t